# 🐛 NSM Debug Notebook (VS Code Local)

**Usage** : Développement et debug rapide en local
**Device** : CPU (pas besoin GPU pour tester)
**Données** : Subset réduit (10 primitives) pour tests rapides
**Production** : Transférer code validé vers NSM_SentenceBERT_Local.ipynb

---

## 🎯 Workflow

1. **Tester ici** : Code, structure, logique
2. **Déboguer** : Breakpoints, variables explorer
3. **Valider** : Résultats sur petit dataset
4. **Transférer** : Copier vers notebook Colab
5. **Exécuter** : Full dataset sur GPU gratuit

In [ ]:
# Setup minimal (local)
import sys
import os

# Path vers données NSM (même repo)
notebooks_path = '/home/stephane/GitHub/Panini/research/semantic-primitives/notebooks'
if notebooks_path not in sys.path:
    sys.path.insert(0, notebooks_path)

from donnees_nsm import NSM_PRIMITIVES, COULEURS_CATEGORIES, CARRES_SEMIOTIQUES, CORPUS_TEST

print(f"✅ Données chargées")
print(f"   Primitives : {len(NSM_PRIMITIVES)}")
print(f"   Carrés : {len(CARRES_SEMIOTIQUES)}")
print(f"   Corpus : {len(CORPUS_TEST)} phrases")

## 🧪 Test 1 : Subset réduit (debug rapide)

**Astuce** : Travailler sur 10 primitives au lieu de 60
- CPU : 2 sec vs 30 sec
- Itération rapide
- Validation logique code

In [ ]:
# Subset pour tests rapides
DEBUG_MODE = True  # Mettre False pour full dataset

if DEBUG_MODE:
    # Prendre 10 primitives variées (2 par catégorie)
    primitives_debug = {
        'JE': NSM_PRIMITIVES['JE'],
        'TU': NSM_PRIMITIVES['TU'],
        'FAIRE': NSM_PRIMITIVES['FAIRE'],
        'ARRIVER': NSM_PRIMITIVES['ARRIVER'],
        'SAVOIR': NSM_PRIMITIVES['SAVOIR'],
        'PENSER': NSM_PRIMITIVES['PENSER'],
        'VOULOIR': NSM_PRIMITIVES['VOULOIR'],
        'SENTIR': NSM_PRIMITIVES['SENTIR'],
        'BON': NSM_PRIMITIVES['BON'],
        'MAL': NSM_PRIMITIVES['MAL']
    }
    print(f"🐛 DEBUG MODE : {len(primitives_debug)} primitives (test rapide)")
else:
    primitives_debug = NSM_PRIMITIVES
    print(f"🚀 PRODUCTION MODE : {len(primitives_debug)} primitives (full)")

primitives_list = list(primitives_debug.items())
primitives_text = [p.forme_francaise for nom, p in primitives_list]
primitives_noms = [nom for nom, p in primitives_list]
primitives_categories = [p.categorie for nom, p in primitives_list]

print(f"\n📊 Dataset :")
for cat in set(primitives_categories):
    count = primitives_categories.count(cat)
    print(f"   {cat:20s} : {count}")

In [ ]:
# Charger modèle SentenceBERT (CPU local)
from sentence_transformers import SentenceTransformer
import torch
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Device : {device}")

if device == 'cpu':
    print("   ⚠️ Mode CPU : lent mais parfait pour debug")
    print("   💡 Pour production : utiliser Colab avec GPU T4 gratuit")

print("\n📥 Chargement modèle...")
start = time.time()
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)
print(f"✅ Chargé en {time.time()-start:.1f}s")

## 🐛 Point de debug : Encodage

**Utilisez breakpoint ici** :
1. Clic gauche sur numéro ligne ci-dessous
2. Point rouge apparaît
3. Exécutez cellule → pause automatique
4. Variables explorer → voir `embeddings`, `primitives_text`
5. Step through → avancer ligne par ligne

In [ ]:
# Encodage avec chronomètre
import numpy as np

print(f"🔢 Encodage {len(primitives_text)} primitives...")
start = time.time()

# POINT DE DEBUG : mettre breakpoint sur ligne suivante
embeddings = model.encode(
    primitives_text,
    batch_size=8,  # Petit batch pour CPU
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

encode_time = time.time() - start

print(f"\n✅ Encodage terminé en {encode_time:.2f}s")
print(f"   Shape : {embeddings.shape}")
print(f"   Dtype : {embeddings.dtype}")
print(f"   Range : [{embeddings.min():.3f}, {embeddings.max():.3f}]")
print(f"   Mean norm : {np.mean(np.linalg.norm(embeddings, axis=1)):.3f}")

# Vérifications sanity check
assert embeddings.shape[0] == len(primitives_text), "Mismatch nombre embeddings"
assert embeddings.shape[1] == 768, "Dimension incorrecte"
assert not np.isnan(embeddings).any(), "NaN détecté dans embeddings"

print("\n✅ Tous les checks passés")

## 🧪 Test clustering rapide

In [ ]:
# Clustering K-means
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

# Mapper catégories
categories_uniques = sorted(set(primitives_categories))
cat_to_label = {cat: i for i, cat in enumerate(categories_uniques)}
labels_true = [cat_to_label[cat] for cat in primitives_categories]

print(f"📊 Clustering K-means (k={len(categories_uniques)})...")
kmeans = KMeans(n_clusters=len(categories_uniques), random_state=42, n_init=10)
labels_pred = kmeans.fit_predict(embeddings)

# Calcul pureté
cm = confusion_matrix(labels_true, labels_pred)
row_ind, col_ind = linear_sum_assignment(-cm)
purete = cm[row_ind, col_ind].sum() / len(labels_true)

# Silhouette
silhouette = silhouette_score(embeddings, labels_pred)

print(f"\n📈 Résultats (DEBUG MODE) :")
print(f"   Pureté : {purete:.3f}")
print(f"   Silhouette : {silhouette:.3f}")

if DEBUG_MODE:
    print("\n⚠️ ATTENTION : Résultats sur subset (10 primitives)")
    print("   Pour résultats finaux : exécuter sur Colab avec 60 primitives")

## 🐛 Debug helpers

In [ ]:
# Fonction debug : inspecter embedding d'une primitive
def debug_embedding(nom_primitive):
    """Affiche stats détaillées d'un embedding"""
    if nom_primitive not in primitives_noms:
        print(f"❌ Primitive '{nom_primitive}' introuvable")
        print(f"   Disponibles : {primitives_noms}")
        return
    
    idx = primitives_noms.index(nom_primitive)
    emb = embeddings[idx]
    
    print(f"🔍 DEBUG : {nom_primitive}")
    print(f"   Texte : {primitives_text[idx]}")
    print(f"   Catégorie : {primitives_categories[idx]}")
    print(f"   Shape : {emb.shape}")
    print(f"   Norm L2 : {np.linalg.norm(emb):.3f}")
    print(f"   Min/Max : [{emb.min():.3f}, {emb.max():.3f}]")
    print(f"   Mean : {emb.mean():.3f}")
    print(f"   Std : {emb.std():.3f}")
    print(f"   Top 5 dims : {np.argsort(np.abs(emb))[-5:]}")

# Test
debug_embedding('JE')

In [ ]:
# Fonction debug : distances entre primitives
def debug_distance(nom1, nom2):
    """Calcule distance cosinus entre 2 primitives"""
    from sklearn.metrics.pairwise import cosine_distances
    
    if nom1 not in primitives_noms or nom2 not in primitives_noms:
        print(f"❌ Primitive(s) introuvable(s)")
        return
    
    idx1 = primitives_noms.index(nom1)
    idx2 = primitives_noms.index(nom2)
    
    emb1 = embeddings[idx1]
    emb2 = embeddings[idx2]
    
    dist = cosine_distances([emb1], [emb2])[0][0]
    similarity = 1 - dist
    
    print(f"🔍 DISTANCE : {nom1} ↔ {nom2}")
    print(f"   Distance cosinus : {dist:.3f}")
    print(f"   Similarité : {similarity:.3f}")
    print(f"   Interprétation : {'Proches' if dist < 0.3 else 'Moyennement proches' if dist < 0.6 else 'Éloignées'}")

# Tests
debug_distance('JE', 'TU')  # Devraient être proches (même catégorie)
print()
debug_distance('JE', 'FAIRE')  # Devraient être éloignées (catégories différentes)

## ✅ Checklist avant transfert Colab

Avant de copier code vers notebook Colab :

- [ ] ✅ Pas d'erreur Python (syntax, import, etc.)
- [ ] ✅ Résultats cohérents sur subset
- [ ] ✅ Pas de NaN dans embeddings
- [ ] ✅ Shape correcte (N, 768)
- [ ] ✅ Silhouette > 0 (au moins)
- [ ] ✅ Variables bien nommées
- [ ] ✅ Code commenté

**Si tout est ✅** → Copier vers `NSM_SentenceBERT_Local.ipynb` et exécuter sur Colab GPU

## 🎯 Prochaines étapes

1. **Local (ce notebook)** : 
   - Tester nouvelle logique
   - Déboguer avec breakpoints
   - Valider sur subset

2. **Colab (NSM_SentenceBERT_Local.ipynb)** :
   - Copier code validé
   - Exécuter sur full dataset (60 primitives)
   - Utiliser GPU T4 gratuit
   - Sauvegarder résultats

3. **Publication** :
   - Analyser résultats
   - Préparer visualisations
   - Écrire paper ACL 2026